# Course 2 : Exercise 1 : Cats vs. Dogs

This week you explored a reduced version of the Cats v Dogs dataset and used it to train a convolutional neural network. You saw that it overfit very quickly, despite great results with the training set. One solution to overfitting is to use more data for both training and validation, and that’s this week’s exercise -- to build a classifier using the full Cats v Dogs dataset of 25k images!

Note again that when loading the images, you might get warnings about EXIF data being missing or corrupt. Don't worry about this -- it is missing data in the images, but it's not visual data that will impact the training.

Let's start building a classifier using the full Cats v Dogs dataset of 25k images.

In [66]:
# ATTENTION: Please do not alter any of the provided code in the exercise. Only add your own code where indicated
# ATTENTION: Please do not add or remove any cells in the exercise. The grader will check specific cells based on the cell position.
# ATTENTION: Please use the provided epoch values when training.

# In this exercise you will train a CNN on the FULL Cats-v-dogs dataset
# This will require you doing a lot of data preprocessing because
# the dataset isn't split into training and validation for you
# This code block has all the required inputs
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd

In [67]:
print(tf.__version__)

2.2.0


In [70]:
#path_cats_and_dogs = f"{getcwd()}/../tmp2/cats-and-dogs.zip"
#shutil.rmtree('/tmp')

local_zip = 'source/cats_and_dogs.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('source')
zip_ref.close()

In [71]:
print(len(os.listdir('source/PetImages/Cat/')))
print(len(os.listdir('source/PetImages/Dog/')))

# Expected Output:
# 1500
# 1500

1500
1500


In [72]:
# Use os.mkdir to create your directories
# You will need a directory for cats-v-dogs, and subdirectories for training
# and testing. These in turn will need subdirectories for 'cats' and 'dogs'
try:
    #YOUR CODE GOES HERE
    os.mkdir('source/cats-v-dogs') 
    os.mkdir('source/cats-v-dogs/training') 
    os.mkdir('source/cats-v-dogs/testing') 
    os.mkdir('source/cats-v-dogs/training/cats') 
    os.mkdir('source/cats-v-dogs/testing/cats') 
    os.mkdir('source/cats-v-dogs/training/dogs') 
    os.mkdir('source/cats-v-dogs/testing/dogs') 
    
except OSError:
    pass

In [73]:
# Write a python function called split_data which takes
# a SOURCE directory containing the files
# a TRAINING directory that a portion of the files will be copied to
# a TESTING directory that a portion of the files will be copied to
# a SPLIT SIZE to determine the portion
# The files should also be randomized, so that the training set is a random
# X% of the files, and the test set is the remaining files
# SO, for example, if SOURCE is PetImages/Cat, and SPLIT SIZE is .9
# Then 90% of the images in PetImages/Cat will be copied to the TRAINING dir
# and 10% of the images will be copied to the TESTING dir
# Also -- All images should be checked, and if they have a zero file length,
# they will not be copied over
#
# os.listdir(DIRECTORY) gives you a listing of the contents of that directory
# os.path.getsize(PATH) gives you the size of the file
# copyfile(source, destination) copies a file from source to destination
# random.sample(list, len(list)) shuffles a list
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    # YOUR CODE STARTS HERE
    source_images = os.listdir(SOURCE)
    size = os.path.getsize(SOURCE)
    train_size = int(1500 * SPLIT_SIZE)
    if size != 0:
        source_images = random.sample(source_images, len(source_images))
        
        for each in source_images[0:train_size]:
            copyfile(os.path.join(SOURCE, each), os.path.join(TRAINING, each))
        for each in source_images[train_size:]:
            copyfile(os.path.join(SOURCE, each), os.path.join(TESTING, each))
    # YOUR CODE ENDS HERE
    
CAT_SOURCE_DIR = "source/PetImages/Cat/"
TRAINING_CATS_DIR = "source/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "source/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "source/PetImages/Dog/"
TRAINING_DOGS_DIR = "source/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "source/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

In [74]:
print(len(os.listdir('source/cats-v-dogs/training/cats/')))
print(len(os.listdir('source/cats-v-dogs/training/dogs/')))
print(len(os.listdir('source/cats-v-dogs/testing/cats/')))
print(len(os.listdir('source/cats-v-dogs/testing/dogs/')))

# Expected output:
# 1350
# 1350
# 150
# 150

1350
1350
150
150


In [75]:
# DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS
# USE AT LEAST 3 CONVOLUTION LAYERS
model = tf.keras.models.Sequential([
    # YOUR CODE HERE
    tf.keras.layers.Conv2D(16, (3, 3), activation = 'relu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

# NOTE:

In the cell below you **MUST** use a batch size of 10 (`batch_size=10`) for the `train_generator` and the `validation_generator`. Using a batch size greater than 10 will exceed memory limits on the Coursera platform.

In [76]:
TRAINING_DIR = os.path.join('source/cats-v-dogs', 'training') #YOUR CODE HERE
train_datagen = ImageDataGenerator(rescale = 1.0/255) #YOUR CODE HERE

# NOTE: YOU MUST USE A BATCH SIZE OF 10 (batch_size=10) FOR THE 
# TRAIN GENERATOR.
# YOUR CODE HERE
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size = 10,
                                                    class_mode = 'binary',
                                                    target_size = (150, 150))

VALIDATION_DIR = os.path.join('source/cats-v-dogs', 'testing') #YOUR CODE HERE
validation_datagen = ImageDataGenerator(rescale = 1.0/255.) #YOUR CODE HERE

# NOTE: YOU MUST USE A BACTH SIZE OF 10 (batch_size=10) FOR THE 
# VALIDATION GENERATOR.
# YOUR CODE HERE
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size = 10,
                                                              class_mode = 'binary',
                                                              target_size = (150, 150))

# Expected Output:
# Found 2700 images belonging to 2 classes.
# Found 300 images belonging to 2 classes.

Found 2700 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [77]:
history = model.fit_generator(train_generator,
                              validation_data=validation_generator,
                              epochs=2,
                              verbose=1)

Epoch 1/2
 55/270 [=====>........................] - ETA: 51s - loss: 0.9096 - accuracy: 0.5382

UnknownError:  OSError: cannot identify image file <_io.BytesIO object at 0x0000023785E72CA8>
Traceback (most recent call last):

  File "C:\Users\User\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\script_ops.py", line 243, in __call__
    ret = func(*args)

  File "C:\Users\User\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\autograph\impl\api.py", line 309, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\User\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 785, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\User\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 801, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\User\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 932, in generator_fn
    yield x[i]

  File "C

In [78]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

# Desired output. Charts with training and validation metrics. No crash :)

NameError: name 'history' is not defined

In [79]:
# Here's a codeblock just for fun. You should be able to upload an image here 
# and have it classified without crashing

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
    # predicting images
    path = '/content/' + fn
    img = image.load_img(path, target_size=(# YOUR CODE HERE))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    if classes[0]>0.5:
        print(fn + " is a dog")
    else:
        print(fn + " is a cat")

SyntaxError: invalid syntax (<ipython-input-79-7e095c5ac888>, line 14)

# Submission Instructions

In [ ]:
# Now click the 'Submit Assignment' button above.

# When you're done or would like to take a break, please run the two cells below to save your work and close the Notebook. This will free up resources for your fellow learners. 

In [ ]:
%%javascript
<!-- Save the notebook -->
IPython.notebook.save_checkpoint();

In [ ]:
%%javascript
IPython.notebook.session.delete();
window.onbeforeunload = null
setTimeout(function() { window.close(); }, 1000);